# 웹 크롤링/스크래핑
## 웹 페이지 추출

In [2]:
PATHFOLDER = 'C:/Users/user/bigdata2022/Python외부데이터수집정제'

### urlib을 이용한 웹 페이지 추출

In [3]:
from urllib.request import urlopen

f = urlopen('http://hanbit.co.kr')

# urlopen() 함수는 HTTPResponse 자료형의 객체를 반환
# 파일 객체이므로 open() 함수로 반환되는 파일 객체처럼 핸들링
type(f)

http.client.HTTPResponse

In [4]:
# read() method로 HTTP 응답 본문(bytes 자료형)을 추출
# HTTP 연결은 자동으로 close 되므로 별도의 close() 함수를 호출
f.read()
f.status

200

In [5]:
f.getheader('Content-Type')

'text/html; charset=UTF-8'

### 문자코드 다루기
- HTTPResponse.read() method로 추출할 수 있는 HTTPResponse 본문은 bytes 자료형
- 문자열(str 자료형)로 다루려면 문자 코드를 지정해서 디코딩 필요
- 최근에는 HTML5의 기본 인코딩 방식인 UTF-8로 전제로 디코딩 가능
- 한국어 사이트를 크롤링 시 여러 가지 인코딩이 혼합되어 있을 수 있으므로 HTTP 헤어를 참조해서 적절한 인코딩 방식으로 디코딩 필요

### HTTP 헤더에서 인코딩 방식 추출
- HTTP 응답의 Content-Type 헤더를 참조하면 해당 페이지의 인코딩 방식을 확인 가능
- 한국어가 포함된 페이지의 일반적인 Content-Type 헤더
    - text/html
    - text/html : charset=UTF-8
    - test/html : charset=EUC-KR
- UTF-8과 EUC-KR이 해당 페이지의 인코딩 방식
- 인코딩이 명시돼 있지 않은 경우 UTF-8 인코딩으로 간주
- HTTPMessage 객체의 get_content_charset() method 사용하여 인코딩 추출 가능

### HTTP 헤더에서 인코딩 방식 추출

In [6]:
import sys
from urllib.request import urlopen

f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')

# HTTP 에더를 기반으로 인코딩 방식을 추출(명시되어 있지 않을 경우 utf-8을 사용)
encoding = f.info().get_content_charset(failobj="utf-8")

#인코딩 방식을 표준 오류애 출력
print('encoding : ', encoding, file=sys.stderr)

# 추출한 인코딩 방식으로 디코딩
text = f.read().decode(encoding)

# 웹 페이지의 내용을 표준 출력에 출력
# print(text)

encoding :  utf-8


### meta 태그에서 인코딩 방식 추출
- 웹 서버 설정에 따라 HTTP 헤더의 Content-Type 인코딩과 실제 사용되고 있는 인코딩 형식 상이
- 브라우저는 HTML 내부의 meta 태그 또는 응답 본문의 바이트열 확인을 통해 최종 인코딩 방식 결정 및 화면 출력
- 디코딩 처리에서 UnicodeDecodeError 발생 시 이러한 방식으로 구현 가능
- HTML meta에 명시되는 인코딩 형식
    - <meta charset="utf-8">
    - <meta http-equiv="Content-Type" content="text/html : charset=EUC_KR">

In [7]:
import re
import sys

from urllib.request import urlopen

f = urlopen('http://www.hanbit.co.kr/store/books/full_book_list.html')
# bytes 자료형의 응답 본문을 일단 변수에 저장
bytes_content = f.read()

# charset은 HTML의 앞부분에 적혀 있는 경우가 많으므로 응답 본문의 앞부분 1024 바이트를 ASCII 문자로 디코딩
# ASCII 범위 이위의 문자는 U+FFFD(REPLACEMENT CHARACTER)로 변환되어 예외가 발생하지 않음
scanned_text = bytes_content[:1024].decode('ascii', errors='replace')

# 디코딩한 문자ㅇㄹ에서 정규 표현식으로 charset 값을 추출
match = re.search(r'charset=["\"]?([\w-]+)', scanned_text)

match.group(1)

'utf-8'

In [8]:
if match :
    encoding = match.group(1)
else :
    # charset이 명시되어 있지 않으면 UTF-8을 사용
    encoding = 'utf-8'

### 웹 페이지에서 데이터 추출
- 정규 표현식을 이용한 스크래핑은 HTML을 단순한 문자열로 취급하여 필요한 정보 추출 마크업되지 않은 웹 페이지도 문자열의 특징을 파악하면 스크래핑 가능
- XML 파서를 이용한 스크래핑은 XML 태그를 분석(파싱)하여 필요한 정보 추출, 블로그 또는 뉴스사이트 정보를 전달하기 위한 RSS와 같이 많은 데이터가 XML형태로 제공, XML 파서를 사용하면 정규 표현식보다 간단하고 효과적으로 필요한 정보 추출 가능
- HTML을 스크래핑할 때는 HTML 전용 파서가 필요, 파이썬의 표준 모듈인 html.parser 모듈을 사용하면 HTML 파싱 가능(복잡한 처리 필요), lxml 등과 같은 라이브러리를 사용하여 HTML 파싱 필요

### 정규 표현식을 이용한 스크래핑

#### 정규식(Regular expressions,, Regex 또는 Regexp)이란?
특정 검색 패턴(ASCII 또는 유니코드 문자의 시퀀스)에 대한 하나 이상의 일치 항목을 검색
검색된 텍스트로부터 정보를 추출하는데 매우 융ㅇ하게 사용가능한 표현식
유효성 검사에서 문자열 파싱 및 대체, 데이터를 다른 형식으로 변환 및 웹 스크래핑에 이르기까지 다양한 응용분야에서 활용
프로그래밍 언너와 텍스트 에디터에 적용 가능

#### RSS(Really Simple Syndication, Rich Site Summary)
뉴스나 블로그 등 업데이트가 빈번한 사이트에서 주로 사용하는 콘텐츠 표현 방식
구독자들에게 업데이트된 정보를 용이하게 제공하기 위해 XML기반으로 정보 펴햔 및 제공
RSS 서브시를 이용하면 업데이트된 정보를 찾기 위해 홈페이지를 일일이 방문하지 않아도 업데이트 될 떄마다 빠르고 편리하게 확인 가능
브라우저에 확장 프로그램으로 제공되기도 함
##### 사이트
    - 기상청 RSS 제공 사이트
        - http://www.weather.go.kr/weather/lifenindustry/service_rss.jsp
    - 서울, 경기도 지역 중기예보 RSS URL
        - http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnid=109

## XML을 이용한 스크래핑

### RSS 스크래핑

In [9]:
TREEPATH = PATHFOLDER + '/rss.xml'

# ElementTree 모듈을 로드
from xml.etree import ElementTree

# parse() 함수로 파일을 읽고 ElementTree 객체를 생성
tree = ElementTree.parse(TREEPATH)

# getroot() method로 XML의 루트 element를 추출
root = tree.getroot()

In [10]:
import pandas as pd

데이터프레임_리스트 = []

for item in root.findall('channel/item/description/body/location/data'):
    # find() method로 element 탐색, text 속성으로 값을 추출
    tm_df = item.find('tmEf').text
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    
    데이터프레임 = pd.DataFrame({
        '일시':[tm_df],
        '최저기온':[tmn],
        '최고기온':[tmx],
        '날씨':[wf],
    })
    데이터프레임_리스트.append(데이터프레임)
날씨정보 = pd.concat(데이터프레임_리스트)
날씨정보

,일시,최저기온,최고기온,날씨
0,2020-06-25 00:00,21,26,흐리고 비
0,2020-06-25 12:00,21,26,흐리고 비
0,2020-06-26 00:00,21,29,흐리고 비
0,2020-06-26 12:00,21,29,구름많음
0,2020-06-27 00:00,22,29,구름많음
...,...,...,...,...
0,2020-06-29 00:00,23,26,흐리고 비
0,2020-06-29 12:00,23,26,흐리고 비
0,2020-06-30 00:00,22,26,흐리고 비
0,2020-07-01 00:00,22,26,흐리고 비


In [11]:
type(날씨정보)

pandas.core.frame.DataFrame

In [12]:
날씨정보.to_csv('날씨정보.csv')
엑셀 = pd.ExcelWriter('날씨정보.xlsx')
날씨정보.to_excel(엑셀, '.', index=False )
엑셀.save()

In [13]:
날씨정보.reset_index(drop=True, inplace=True)

In [14]:
날씨정보.to_json('날씨정보.json')

In [15]:
import sqlite3
from pandas.io import sql
import os

In [16]:
with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
    try:
        날씨정보.to_sql(name = 'WEATHER_INFO', con = con, index = False, if_exists='append') 
        #if_exists : {'fail', 'replace', 'append'} default : fail
    except Exception as e:
        print(str(e))
    
    query = 'SELECT * FROM WEATHER_INFO'
    데이터프레임1 = pd.read_sql(query, con = con)

In [17]:
엑셀 = pd.ExcelWriter('날씨정보2.xlsx')
데이터프레임1.to_excel(엑셀, '.', index=False )
엑셀.save()

In [18]:
df = pd.read_excel('날씨정보2.xlsx')

In [19]:
df

,일시,최저기온,최고기온,날씨
0,2020-06-25 00:00,21,26,흐리고 비
1,2020-06-25 12:00,21,26,흐리고 비
2,2020-06-26 00:00,21,29,흐리고 비
3,2020-06-26 12:00,21,29,구름많음
4,2020-06-27 00:00,22,29,구름많음
...,...,...,...,...
4792,2020-06-29 00:00,23,26,흐리고 비
4793,2020-06-29 12:00,23,26,흐리고 비
4794,2020-06-30 00:00,22,26,흐리고 비
4795,2020-07-01 00:00,22,26,흐리고 비


### CSV (Comma-Seperated Values)
하나의 레코드를 한 라인에 저장하는 텍스트 포맷 파일
각 라인의 컬럼값은 쉼표(콤마) 등의 구분자를 사용
데이터 내에 쉽표 등의 구분자 사용하는 문자가 포함되어 있다면 파싱 시 문제가 발생
데이터 내에 쉼표 등의 문자가 포함되어 있는 경우 TSV(Tab-Seperated Values) 형식 사용 필요

## CSV 형식으로 저장

In [20]:
import csv

with open('top_cities.csv', 'w', newline='', encoding='utf-8') as f :
    # 첫 번째 매개변수에 파일 객체
    # 두 번째 배개변수에 필드명 리스트를 지정
    writer = csv.DictWriter(f, ['rank', 'city', 'population'])
        # 첫 번째 줄에 헤더를 입력
    writer.writeheader()
    # writerows()로 여러 개의 데이터를 딕셔너리 형태로 작성
    writer.writerows([
        {'rank':1,'city':'상하이','population':24150000},
        {'rank':2,'city':'카라치','population':23500000},
        {'rank':3,'city':'베이징','population':21516000},
        {'rank':4,'city':'텐진','population':14722100},
        {'rank':5,'city':'이스탄불','population':14160467},
    ])

In [21]:
import chardet

char_dic = chardet.detect(open('top_cities.csv', 'rb').read())
char_dic['encoding']

'utf-8'

## JSON 형식으로 저장 

In [22]:
import json

cities = [
    {'rank': 1, 'city':'상하이', 'population': 24150000},
    {'rank': 2, 'city':'카라치', 'population': 23500000},
    {'rank': 3, 'city':'베이징', 'population': 21516000},
    {'rank': 4, 'city':'텐진', 'population': 14722100}, 
    {'rank': 5, 'city':'이스탄불', 'population':14160467},
]

with open('top_cities.json', 'w') as fw:
    json.dump(cities, fw)
    
with open('top_cities.json', 'r') as fr:
    json_file = json.load(fr)
    print(json_file)

[{'rank': 1, 'city': '상하이', 'population': 24150000}, {'rank': 2, 'city': '카라치', 'population': 23500000}, {'rank': 3, 'city': '베이징', 'population': 21516000}, {'rank': 4, 'city': '텐진', 'population': 14722100}, {'rank': 5, 'city': '이스탄불', 'population': 14160467}]


## SQLite3 DBMS로 저장

In [23]:
import pandas as pd
import sqlite3
from pandas.io import sql
import os

DB_NAME = 'top_cities.db'
TABLE_NAME = 'TOP_CITIES'

def db_save(df, db_name, table_name):
    with sqlite3.connect(db_name) as con:
        try:
            df.to_sql(name = table_name, con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(df), '건 저장완료..')

        
def db_select(db_name, table_name):
    with sqlite3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df

    
def db_delete(db_name, table_name):
    with sqlite3.connect(db_name) as con: 
        try:
            cur = con.cursor()
            sql = 'DELETE FROM {}'.format(table_name)
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [24]:
# db_delete(DB_NAME, TABLE_NAME)

In [25]:
top_cites = pd.read_csv('top_cities.csv')
db_save(top_cites, DB_NAME, TABLE_NAME)

5 건 저장완료..


In [26]:
df = db_select(DB_NAME, TABLE_NAME)
df

,rank,city,population
0,1,상하이,24150000
1,2,카라치,23500000
2,3,베이징,21516000
3,4,텐진,14722100
4,5,이스탄불,14160467
5,1,상하이,24150000
6,2,카라치,23500000
7,3,베이징,21516000
8,4,텐진,14722100
9,5,이스탄불,14160467


# Python Scraper

In [27]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [28]:
def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    return html


def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url': [url], 'title': [title]}))
    return pd.concat(books)


def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [29]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

df = scrape(html)
df.reset_index(drop=True, inplace=True)
df2 = save('books.db', df)
df2

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 무시무시 마녀가 이사 와!
1,http://www.hanbit.co.kr/store/books/look.php?p...,"받침 없는 한글 동화 : 도, 도, 도깨비다!"
2,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 바쁘다 바빠 너구리 바빠
3,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 도깨비 파자마 파티
4,http://www.hanbit.co.kr/store/books/look.php?p...,받침 없는 한글 동화 : 4권 세트
...,...,...
295,http://www.hanbit.co.kr/store/books/look.php?p...,"회사에서 바로 통하는 실무 엑셀+파워포인트+워드&한글(모든 버전 사용 가능, 개정판)"
296,http://www.hanbit.co.kr/store/books/look.php?p...,혼자 공부하는 얄팍한 코딩 지식
297,http://www.hanbit.co.kr/store/books/look.php?p...,똑똑한 코드 작성을 위한 실전 알고리즘
298,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, UI/UX 디자인 이론과 실무 with 어도비 XD"


# Scape by LXML

In [31]:
!pip install lxml
!pip install cssselect

In [32]:
import lxml.html

DPPATH = PATHFOLDER + '/dp.html'

# HTML 파일을 읽어 들이고, getroot() 메서드로 HtmlElement 객체 생성
tree = lxml.html.parse(DPPATH)
html = tree.getroot()

# cssselect() 메서드로 a 요소의 리스트를 추출 및 반복 수행
for a in html.cssselect('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)

#gnb None
#top_search None
#container None
https://www.hanbit.co.kr/index.html None
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.html Channel.H
https://www.hanbit.co.kr/store/store_submain.html STORE
https://www.hanbit.co.kr/support/help_info.html SUPPORT
https://www.hanbit.co.kr/event/current/current_event_list.

# Scrape by BS4

In [33]:
# !pip install beautifulsoup4

In [34]:
from bs4 import BeautifulSoup

In [35]:
PATHBOOKLIST = PATHFOLDER + '/data/full_book_list.html'

# HTML 파일을 읽어 들이고 BeautifulSoup 객체를 생성
with open(PATHBOOKLIST, encoding="utf-8") as f:
    soup = BeautifulSoup(f, 'html.parser')
    
# find_all() 메서드로 a 요소를 추출 및 반복 수행
for a in soup.find_all('a'):
    # href 속성과 글자를 추출합니다.
    print(a.get('href'), a.text)

#gnb 메뉴 바로가기
#top_search 검색 및 카테고리 바로가기
#container 본문 바로가기
https://www.hanbit.co.kr/index.html HOME
https://www.hanbit.co.kr/media/ 한빛미디어
https://www.hanbit.co.kr/academy/ 한빛아카데미
https://www.hanbit.co.kr/biz/ 한빛비즈
https://www.hanbit.co.kr/life/ 한빛라이프
https://www.hanbit.co.kr/edu/ 한빛에듀
https://www.hanbit.co.kr/realtime/ 리얼타임
https://www.hanbit.co.kr/textbook/ 한빛정보교과서
https://www.hanbit.co.kr/rent/ 한빛대관서비스
https://www.hanbit.co.kr/member/login.html 로그인
https://www.hanbit.co.kr/member/member_agree.html 회원가입
https://www.hanbit.co.kr/myhanbit/myhanbit.html 마이한빛
https://www.hanbit.co.kr/myhanbit/cart.html 장바구니
https://www.hanbit.co.kr/publisher/foreignrights.html?lang=e ENGLISH
https://www.hanbit.co.kr/index.html 한빛출판네트워크
https://www.hanbit.co.kr/brand/brand_submain.html BRAND
https://www.hanbit.co.kr/channel/channel_submain.html Channel.H
https://www.hanbit.co.kr/store/store_submain.html STORE
https://www.hanbit.co.kr/support/help_info.html SUPPORT
https://www.hanbit.co.kr/event/current/cur